In [ ]:
!pip install transformers
!pip install nltk/


     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 890kB 10.8MB/s 
     |████████████████████████████████| 1.1MB 19.3MB/s 
     |████████████████████████████████| 3.0MB 18.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e3f073a4819bed3299ad6c08370e9b29a1bc00342fd84ad174632d23754adef7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: Invalid requirement: 'nltk/'
Hint: It looks like a path. File 'nltk/' does not exist.


In [ ]:
import os
import io
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/train/text

/content/drive/My Drive/MeasEval-main/data/train/train/text


In [ ]:
files=os.listdir()

In [ ]:
cd ..

/content/drive/My Drive/MeasEval-main/data/train/train


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join('text',i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join('tsv',i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
print(len(NERdata))

1370


In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
NERdf.head(5)

,Id,Text,Entity
0,S0006322312001096-1221,"We constructed Framingham CVD, CHD, and stroke...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,S0006322312001096-1221,"Crude, age- and sex-adjusted, and multivariabl...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,S0006322312001096-1221,"In the analysis of GHQ symptoms, the sample wa...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,S0006322312001096-1221,For depressive symptoms defined by CES-D and a...,"[[], [], [MeasuredEntity], [MeasuredEntity], [..."
4,S0167610513002729-1127,The situation for the 45° case is more interes...,"[[], [], [], [], [], [Quantity], [Quantity], [..."


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [ ]:
len(textlabels[1])

512

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
import numpy as np

In [ ]:
train_seq = np.array(inputs['input_ids'])
train_mask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"Quantity":0, "MeasuredEntity": 1, "MeasuredProperty": 2, "Qualifier": 3}

In [ ]:
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),4));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
train_seq, val_seq, train_mask, val_mask, train_y, val_y = train_test_split(train_seq, train_mask, onehot_labels, test_size=0.1, random_state=42)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(train_y))
val_data = TensorDataset(torch.from_numpy(val_seq), torch.from_numpy(val_mask),torch.from_numpy(val_y))

In [ ]:
batch_size = 38
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
for param in model.parameters():
  param.requires_grad = False
    
    

In [ ]:
# params = list(model.named_parameters())

# for p in params:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))



In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim, n_layers,dropout=drop_prob,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      # self.r1=nn.ReLU()
      # self.fc2 = nn.Linear(256,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      embed, cls_hs = self.bert(sent_id, attention_mask=mask)
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      # x = self.r1(x)
      # x = self.fc2(x)
      x = self.sigmoid(x)

      return x

In [ ]:
device = torch.device("cpu")

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 2,4)
bert_model = bert_model.to(device)


In [ ]:
lr = 0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(bert_model.parameters(),lr = 0.005)

In [ ]:
epochs = 20
for e in range(0,epochs):
  
  bert_model.train()
  i=0
  train_loss=0
  for seq, mask, y in train_loader:
    bert_model.zero_grad()
    print(seq.shape,y.shape)
    y_pred = bert_model(seq.to(device), mask.to(device))
    loss = criterion(y_pred,y.to(device).float())
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*batch_size
    if(i%5==0):
      print("Epoch-{}/{} Iterations-{} loss-{}".format(e+1,epochs,i+1,loss.item()))
    i+=1
  
  

  val_loss=0
  for seq, mask, y in val_loader:
    bert_model.zero_grad()
    y_pred = bert_model(seq.to(device), mask.to(device))
    loss = criterion(y_pred,y.to(device).float())
    val_loss += loss.item()*batch_size
    
    i+=1
  
  print("Epoch-{}/{} train_loss-{} Val_loss-{}".format(e+1,epochs,train_loss/len(train_loader),val_loss/len(val_loader)))


torch.save(bert_model,'/content/drive/My Drive/Model/bert_last_layer.pt')
  
    
  
  
  

torch.Size([38, 512]) torch.Size([38, 512, 4])
Epoch-1/20 Iterations-1 loss-0.7055626511573792
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
Epoch-1/20 Iterations-6 loss-0.021674171090126038
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])
torch.Size([38, 512]) torch.Size([38, 512, 4])


In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in val_loader:
    #bert_model2.zero_grad()
    #bert_model2.eval()
    #y_pred = bert_model2(seq.to(device), mask.to(device))
    #np_out1 = y_pred.cpu().data.numpy()

    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out2 = y_pred.cpu().data.numpy()

    np_out = (np_out2)#+np_out1)/2
    np_act = y.cpu().data.numpy()
    #print(np_out)
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        #fuck = np.argmax(np_out[i,j,:])
        #np_out[i,j,fuck]=1
        for k in range(1,np_out.shape[2]):
          if np_out[i][j][k]>=0.2:
            #if np_out[i,j,0]==1:
              #np_out[i,j,0]=0
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0



    #for i in range(np_out.shape[0]):
    #  for j in range(np_out.shape[1]):
    #    for k in range(np_out.shape[2]):
    #      if np_out[i][j][k]>=0.3:
    #        np_out[i][j][k]=1
    #      else:
    #        np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,1:])==1 or np.max(np_act[i,j,1:])==1:
          if np.max(np_out[i,j,1:])==1 and np.max(np_act[i,j,1:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(1,np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

# precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
# F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------NER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
# print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
# print("F1 score:-"+str(F1))
    

In [ ]:
torch.save(bert_model,'/content/drive/My Drive/Model/akash_lastlayer_1lstm_sh.pt')